## 희소행렬

- BOW의 Vectorization 모델은 너무 많은 0 값이 메모리 공간에 할당되어 많은 메모리 공간이 필요하며 또한 연산시에도 데이터 엑세스를 위한 많은 시간이 소모 됩니다.

## 희소 행렬의 저장 변환 형식

- COO 형식:Coordinate(좌표) 방식을 의미하며 0이 아닌 데이터만 별도의 배열(Array)에 저장하고 그 데이터를 가리키는 행과 열의 위치를 별도의 배열로 저장하는 방식

- CSR 형식 : COO 형식이 위치 배열값을 중복적으로 가지는 문제를 해결한 방식. 일반적으로 CSR 형식이 COO보다 많이 사용됨

- 파이썬에서는 희소 행렬을 COO, CSR 형식으로 변환하기 위해서 Scipy의 coo_matrix(), csr_matrix() 함수를 이용합니다.

## 뉴스 그룹 분류
### 데이터 로딩과 데이터 구성 확인

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)
len(news_data.target_names), pd.Series(news_data.target).shape

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


(20, (18846,))

In [4]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

## 학습과 테스트용 데이터 생성

In [5]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용(Train) 데이터만 추출, remove=('headers', 'footers', 'quotes)로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target
print(type(X_train))

# subset='test'으로 테스트(Test) 데이터만 추출, remove=('headers', 'footers', 'quotes)로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), random_state=156)
X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0} , 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

<class 'list'>
학습 데이터 크기 11314 , 테스트 데이터 크기 7532


## Count 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

- 학습 데이터에 대해 fit()된 CountVectorizer를 이용해서 테스트 데이터를 피처 벡터화 해야함.
- 테스트 데이터에서 다시 CountVectorrizer의 fit_transform()을 수행하거나 fit()을 수행하면 안됨.
- 이는 이렇게 테스트 데이터에서 fit()을 수행하게 되면 기존 학습된 모델에서 가지는 feature의 갯수가 달라지기 때문임.


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용하여 테스트 데이터를 feature extraction 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape: ', X_train_cnt_vect.shape, X_test_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape:  (11314, 101631) (7532, 101631)


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용하여 학습/예측/평가 수행.
lr_clf=LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression 의 예측 정확도는 {0: .3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 의 예측 정확도는  0.608


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## TF-IDF 피쳐 변환과 머신러닝 학습/예측/평가

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경하여 Feature Vectorization 적용
tfidf_vect = TfidfVectorizer(stop_words='english')

# LogisticRegression을 이용하여 학습/예측/평가 수행.
lr_clf=LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0: .3f}'.format(accuracy_score(y_test,pred)))

TF-IDF Logistic Regression 의 예측 정확도는  0.608


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경하여 피쳐 벡터화

In [9]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경하여 Feature Vectorization 적용.
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)
print(X_train_tfidf_vect)


lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0: .3f}'.format(accuracy_score(y_test,pred)))

  (0, 886781)	0.08730056470975082
  (0, 876926)	0.1498813909180581
  (0, 759360)	0.1498813909180581
  (0, 759324)	0.07624926265726012
  (0, 662604)	0.14357775749331866
  (0, 526899)	0.1498813909180581
  (0, 526477)	0.14357775749331866
  (0, 526386)	0.07408829261147588
  (0, 509693)	0.1498813909180581
  (0, 509638)	0.08937653359823874
  (0, 447271)	0.2997627818361162
  (0, 447252)	0.18004881075949486
  (0, 432223)	0.1498813909180581
  (0, 432222)	0.1248599860746207
  (0, 430081)	0.1498813909180581
  (0, 430059)	0.07805484780993284
  (0, 409267)	0.1498813909180581
  (0, 387254)	0.14357775749331866
  (0, 387106)	0.12649799155244668
  (0, 334045)	0.1498813909180581
  (0, 334036)	0.09726538238916144
  (0, 298002)	0.1498813909180581
  (0, 297933)	0.1498813909180581
  (0, 281444)	0.1498813909180581
  (0, 281399)	0.08265551475782179
  :	:
  (11313, 284199)	0.07014779013700947
  (11313, 277470)	0.0986024304666684
  (11313, 265919)	0.1089582154060023
  (11313, 265908)	0.08618607373905401
  (1131

### GridSearchCV로 LogisticRegression C 하이퍼 파라미터 튜닝

In [12]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행, CV는 3 Fold셋으로 설정
params = { 'C': [0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1 )
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameters : ', grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가.
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
 

Logistic Regression best C parameters :  {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.701


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### 사이킷런 파이프라인(Pipeline) 사용

In [16]:
from sklearn.pipeline import Pipeline

# TfidVectorizer 객체를 tfidf_vect 객체명으로, LogisticRegresssion객체를 lr_clf 객체명으로 생성하는 Pipeline 생성
pipeline = Pipeline([
                     ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
                     ('lr_clf', LogisticRegression(C=10))
])

# 별도의 TfidVectorizer 객체의 fit_transform()과 LogisticRegression의 fit(), predict()가 필요 없음,
# pipeline의 fit()과 predict() 만으로 한꺼번에 Feature Vectorization과 ML, 학습/예측이 가능.
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701
